# Проект "Прогнозирование оттока клиентов ТелеДом"

## Цель проекта
Разработка модели машинного обучения для прогнозирования оттока клиентов оператора связи "ТелеДом" с целью своевременного предложения персональных условий и удержания абонентов.

## Описание данных

### Источники данных
База данных SQLite, содержащая 4 таблицы:
1. `contract` - информация о договорах
2. `personal` - персональные данные клиентов
3. `internet` - данные об интернет-услугах
4. `phone` - данные о телефонных услугах

## Таблицы и поля

### Таблица `contract` (Договоры)
- `customerID` - Уникальный идентификатор клиента
- `BeginDate` - Дата начала действия договора
- `EndDate` - Дата расторжения договора (NULL для действующих клиентов)
- `Type` - Периодичность оплаты: "Ежемесячно" или "Ежегодно"
- `PaperlessBilling` - Использование электронных счетов (Да/Нет)
- `PaymentMethod` - Способ оплаты: "Электронный чек", "Кредитная карта" и др.
- `MonthlyCharges` - Размер ежемесячного платежа
- `TotalCharges` - Общая сумма выплат клиента

### Таблица `personal` (Персональные данные)
- `customerID` - Уникальный идентификатор клиента
- `gender` - Пол клиента
- `SeniorCitizen` - Пенсионный статус
- `Partner` - Наличие супруга/супруги
- `Dependents` - Наличие детей

### Таблица `internet` (Интернет-услуги)
- `customerID` - Уникальный идентификатор клиента
- `InternetService` - Тип подключения
- `OnlineSecurity` - Подписка на интернет-безопасность
- `OnlineBackup` - Подписка на облачное резервное копирование
- `DeviceProtection` - Антивирус
- `TechSupport` - Техническая поддержка
- `StreamingTV` - Стриминговое ТВ
- `StreamingMovies` - Каталог фильмов

### Таблица `phone` (Телефонные услуги)
- `customerID` - Уникальный идентификатор клиента
- `MultipleLines` - Подключение телефона к нескольким линиям одновременно.

## Особенности данных
- Данные актуальны на 1 февраля 2020 года

## Задачи анализа
1. Определить ключевые факторы оттока клиентов
2. Построить прогнозную модель ухода абонентов
3. Выделить сегменты клиентов с высоким риском оттока
4. Разработать рекомендации по удержанию клиентов

In [1]:
import urllib.request
import os
import os.path
from tabulate import tabulate
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [2]:
# объявление констант
DB_PATH = "data/"
DB_FILE = "ds-plus-final.db"
DB_URL = "https://code.s3.yandex.net/data-scientist/ds-plus-final.db"
DB_TABLE_NAMES = ['contract', 'personal', 'internet', 'phone']
DB_TABLE_DICT = {
    "contract": [
        "customerID",
        "BeginDate",
        "EndDate",
        "Type",
        "PaperlessBilling",
        "PaymentMethod",
        "MonthlyCharges",
        "TotalCharges"
    ],
    "personal": [
        "customerID",
        "gender",
        "SeniorCitizen",
        "Partner",
        "Dependents"
    ],
    "internet": [
        "customerID",
        "InternetService",
        "OnlineSecurity",
        "OnlineBackup",
        "DeviceProtection",
        "TechSupport",
        "StreamingTV",
        "StreamingMovies"
    ],
    "phone": [
        "customerID",
        "MultipleLines"
    ]
}                  
RANDOM_STATE = 110825
TARGET_METRIC = .85
TARGET = "EndDate"

# 1. Первичный осмотр данных

In [3]:
# создаем каталог для хранения БД
if not os.path.exists(DB_PATH):
    try:
        os.makedirs(DB_PATH)
        print(f"Каталог {DB_PATH} создан")
    except Exception as err:
        print(f"Ошибка при создании каталога 'data': {err}")

In [4]:
# загружаем файл с БД
if not os.path.exists(DB_PATH+DB_FILE):
    try:
        urllib.request.urlretrieve(DB_URL, DB_PATH+DB_FILE)
        print(f"База данных {DB_FILE} загружена")
    except Exception as err:
        print(f"Ошибка при попытке загрузки базы данных {err}")
else:
    print(f"База данных {DB_FILE} уже загружена")

База данных ds-plus-final.db уже загружена


In [5]:
# подключаемся к БД
engine = create_engine(f'sqlite:///{DB_PATH+DB_FILE}', echo=False) 

### TableGetInfo

Класс для проверки структуры и анализа данных в таблицах базы данных.

### Основной функционал

- Валидация структуры таблиц (соответствие ожидаемым столбцам)
- Анализ метаинформации (типы данных, NULL-значения)
- Подсчет уникальных значений
- Просмотр образцов данных

### Конструктор класса
`__init__(self, engine, tables_dict=None)` - Инициализирует анализатор с подключением к БД и словарем таблиц.

### Внутренние методы

`_get_table_info(table_name)` - Получает метаинформацию о структуре указанной таблицы.

`_check_null_values(table_name)` - Подсчитывает количество NULL-значений для каждого столбца таблицы.

`_get_table_size(table_name)` - Считает общее количество строк в указанной таблице.

`_unique_counts(table_name, columns)` - Вычисляет количество уникальных значений для указанных столбцов.

`_validate_tables()` - Основной метод валидации, проверяет все таблицы из словаря.

### Публичные методы

`print_info()` - Выводит форматированный отчет о структуре таблиц и их данных.

`show_data(limit=5)` - Отображает первые N строк из каждой таблицы для предварительного просмотра.

In [7]:
class TableGetInfo:
    def __init__(self, engine, tables_dict=None):
        if not tables_dict:
            raise ValueError('Словарь tables_dict должен быть указан и не может быть пустым')
        self.engine = engine
        self.tables_dict = tables_dict
        self.results = {}
        self._validate_tables()
    
    def _get_table_info(self, table_name):
        try:
            query = text(f"PRAGMA table_info({table_name})")
            return pd.read_sql_query(query, con=self.engine)
        except SQLAlchemyError as e:
            print(f"Ошибка при получении информации о таблице {table_name}: {e}")
            return None
            
    def _check_null_values(self, table_name):
        null_counts = {}
        try:
            for column in self.tables_dict[table_name]:
                query = text(f"SELECT COUNT(*) FROM {table_name} WHERE {column} IS NULL")
                null_count = pd.read_sql_query(query, self.engine).iloc[0, 0]
                if null_count > 0:
                    null_counts[column] = null_count
        except SQLAlchemyError as e:
            print(f"Ошибка при проверке NULL-значений в {table_name}.{column}: {e}")
        return null_counts
        
    def _get_table_size(self, table_name):
        query = text(f'SELECT COUNT(*) FROM {table_name}')
        try:
            return pd.read_sql_query(query, con=self.engine).iloc[0,0]
        except SQLAlchemyError as e:
            print(f"Ошибка при выполнении запроса: {e}")
            return None
            
    def _unique_counts(self, table_name, columns):
        value_counts = {}
        for col in columns:
            query = text(f"SELECT COUNT(DISTINCT {col}) FROM {table_name}")
            value_counts[col] = pd.read_sql_query(query, engine).iloc[0,0]
        return value_counts
        
    def _validate_tables(self):        
        for table_name, expected_columns in self.tables_dict.items():
            actual_info = self._get_table_info(table_name)
            actual_columns = actual_info['name'].tolist()
            missing_in_db = set(expected_columns) - set(actual_columns)
            extra_in_db = set(actual_columns) - set(expected_columns)
            self.results[table_name] = {
                'status': 'ok' if not missing_in_db and not extra_in_db else 'mismatch',
                'table_size': self._get_table_size(table_name),
                'actual_columns': actual_columns,
                'value_counts': self._unique_counts(table_name, actual_columns),
                'missing_columns': list(missing_in_db),
                'extra_columns': list(extra_in_db),
                'column_types': dict(zip(actual_info['name'], actual_info['type'])),
                'null_values': self._check_null_values(table_name)
            }

    def print_info(self):
        for table_name, data in self.results.items():
            print(f"\nТаблица: {table_name}")
            print("-" * 50)
            
            if data['status'] == 'ok':
                print("✅ Структура полностью соответствует ожидаемой")
            else:
                if data['missing_columns']:
                    print(f"❌ Отсутствующие столбцы: {', '.join(data['missing_columns'])}")
                if data['extra_columns']:
                    print(f"❌ Лишние столбцы в БД: {', '.join(data['extra_columns'])}")

            print(f"\nКоличество строк: {data['table_size']}")
          
            if data['null_values']:
                print("\nПропущенные значения (NULL):")
                for col, count in data['null_values'].items():
                    print(f"  - {col}: {count} пропусков")
            else:
                print("\nПропусков(NULL) нет")
                    
            print("\nПроверка столбцов, типов и уникальных значений:")
            for col in data['actual_columns']:
                status = "🔴" if col in data['missing_columns'] or col in data['extra_columns'] else "🟢"
                col_type = data['column_types'].get(col)
                value_counts = data['value_counts'].get(col)
                print(f"{status} {col}: {col_type}: {value_counts}")            
            print("-" * 50)

    def show_data(self, limit=5):
        for table_name in self.tables_dict.keys():
            try:
                query = text(f"SELECT * FROM {table_name} LIMIT {limit}")
                df = pd.read_sql_query(query, self.engine)
                
                print(f"\n{'-'*50}")
                print(f"ТАБЛИЦА: {table_name} (первые {limit} записей)")
                print('-'*50)
                print(tabulate(df, headers='keys', tablefmt='psql', showindex=False))

            except Exception as e:
                print(f"\nОшибка при обработке таблицы {table_name}: {str(e)}")

In [8]:
# инициализация 
analyzer = TableGetInfo(engine=engine, tables_dict=DB_TABLE_DICT)

In [9]:
# запуск валидаци таблиц БД
analyzer.print_info()


Таблица: contract
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Количество строк: 7043

Пропусков(NULL) нет

Проверка столбцов, типов и уникальных значений:
🟢 customerID: TEXT: 7043
🟢 BeginDate: TEXT: 77
🟢 EndDate: TEXT: 67
🟢 Type: TEXT: 3
🟢 PaperlessBilling: TEXT: 2
🟢 PaymentMethod: TEXT: 4
🟢 MonthlyCharges: TEXT: 1585
🟢 TotalCharges: TEXT: 6658
--------------------------------------------------

Таблица: personal
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Количество строк: 7043

Пропусков(NULL) нет

Проверка столбцов, типов и уникальных значений:
🟢 customerID: TEXT: 7043
🟢 gender: TEXT: 2
🟢 SeniorCitizen: TEXT: 2
🟢 Partner: TEXT: 2
🟢 Dependents: TEXT: 2
--------------------------------------------------

Таблица: internet
--------------------------------------------------
✅ Структура полностью соответствует ожидаемой

Количество строк: 5517

Пропусков(NULL) нет

Проверка столбц

In [10]:
analyzer.show_data()


--------------------------------------------------
ТАБЛИЦА: contract (первые 5 записей)
--------------------------------------------------
+--------------+-------------+-----------+----------------+--------------------+---------------------------+------------------+----------------+
| customerID   | BeginDate   | EndDate   | Type           | PaperlessBilling   | PaymentMethod             |   MonthlyCharges |   TotalCharges |
|--------------+-------------+-----------+----------------+--------------------+---------------------------+------------------+----------------|
| 7590-VHVEG   | 2020-01-01  | No        | Month-to-month | Yes                | Electronic check          |            29.85 |          31.04 |
| 5575-GNVDE   | 2017-04-01  | No        | One year       | No                 | Mailed check              |            56.95 |        2071.84 |
| 3668-QPYBK   | 2019-10-01  | No        | Month-to-month | Yes                | Mailed check              |            53.85 |        

### Вывод по разделу:
1. **Проведенная работа**:
   - Реализован класс `TableGetInfo` для анализа структуры БД
   - Настроена проверка соответствия ожидаемой и фактической структуры таблиц
   - Реализованы методы для проверки:
     * Наличия/отсутствия столбцов
     * Количества NULL-значений
     * Уникальных значений в столбцах
     * Общего размера таблиц

In [11]:
#engine.dispose()